In [1]:
import numpy as np
from sklearn.decomposition import PCA
import pickle
import json

# Load original 512D embeddings
with open("face_embeddings.json", "r") as f:
    data = json.load(f)

embeddings = np.array(data["embeddings"])

# Create and fit PCA model to reduce from 512 to 128 dimensions
pca = PCA(n_components=128)
pca.fit(embeddings)

# Save using your current environment
with open("pca_512_to_128.pkl", "wb") as f:
    pickle.dump(pca, f)

print("✅ PCA model successfully saved!")


✅ PCA model successfully saved!


In [ ]:
from modules.pca_reducer import load_pca_model

# Load PCA
pca = load_pca_model("pca_512_to_128.pkl")

# Assume you have: `embedding_512` from InsightFace
embedding_128 = pca.transform([embedding_512])[0]


In [2]:
import json
import numpy as np
from modules.pca_reducer import load_pca_model

# File paths
input_file = "face_embeddings.json"
output_file = "face_embeddings_128.json"
pca_model_path = "pca_512_to_128.pkl"

# Load 512D embeddings
with open(input_file, "r") as f:
    data = json.load(f)
    embeddings_512 = np.array(data["embeddings"])
    labels = data["labels"]

# Load trained PCA model
pca = load_pca_model(pca_model_path)

# Convert to 128D
embeddings_128 = pca.transform(embeddings_512)

# Save to new JSON
output_data = {
    "embeddings": embeddings_128.tolist(),
    "labels": labels
}

with open(output_file, "w") as f:
    json.dump(output_data, f)

print(f"[DONE] Converted {len(embeddings_128)} embeddings to 128D and saved to {output_file}")


[INFO] Loaded trained PCA for 128D reduction.
[DONE] Converted 5004 embeddings to 128D and saved to face_embeddings_128.json


In [5]:
import numpy as np

# Load all float32 values from the .dat file
data = np.fromfile("neuralhash_128x96_seed1.dat", dtype=np.float32)

# Save to text file
np.savetxt("projection_matrix.txt", data, fmt="%.8f")

print(f"✅ Saved {data.size} values to 'projection_matrix.txt'")


✅ Saved 12320 values to 'projection_matrix.txt'
